## part 7: aggregate score, make recommendation!

In [1]:
import cPickle as pickle

import numpy as np
import pandas as pd
import random
% matplotlib inline

import ipywidgets as widgets

from sklearn.preprocessing import MinMaxScaler

# gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim import similarities

pd.set_option('max_colwidth', -1)

C:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## load data from the various sources

#### Dramalist metadata

In [2]:
# load dramalist drama metadata
metadata = pickle.load(open('dramalist_info_ordered.pkl', 'rb'))
print metadata.shape
metadata.head(20)

(386, 14)


,index,actors,aired_date,directors,ep_dur,ep_no,genres,score,screenwriters,showname,synopsis,tags,image_index,year
0,0,"[Kim So Hyun, Sun Woong, Min Hoo, Hong Yoon Hwa, Kang Hyun Joong, Nam Tae Boo]","Aug 16, 2012 - Aug 30, 2012",NaN,60,3,"[Friendship, Comedy, Romance, School, Youth]",7.5,NaN,ma boy,"Geu Rim is a student at an arts high school who dreams of becoming a pop singer. At school She gets a new roommate, Irene who’s actually a boy. Irena’s persona is a pretty Female CF star and his actual name is Hyun Woo.","[Cross-dressing, Cohabitation, High School, Mini Drama]",0,2012
1,1,"[Kim So Yeon, Jung Kyung Ho, Yoon Hyun Min, Gong Hyun Joo, Jo Eun Ji, Uhm Hyo Sub]","Apr 3, 2015 - May 23, 2015",NaN,60,16,"[Comedy, Romance, Drama, Melodrama]",8.4,NaN,falling for innocence,"Min-Ho's father ran a large company, but his death led to his uncle taking over the company and to Min-Ho's growing into a cutthroat investor. But then his personality changes when the merciless corporate head hunter undergoes a heart transplant surgery. Thanks to his new heart, Min-Ho begins to have warm feelings and he falls in love with a woman named Soon-Jung.","[Love Triangle, Rich Male Lead, Illness, Boy Chases Girl First]",1,2015
2,2,"[Yoo Ji Tae, Woo Do Hwan, Ryu Hwa Young, Jo Jae Yun, Kim Hye Sung, Hong Soo Hyun]","Oct 11, 2017 - Nov 30, 2017",NaN,60,16,"[Thriller, Romance, Crime, Investigation]",8.5,NaN,mad dog,"A realistic drama about the team “Mad Dog”, which catches scam artists that commit insurance fraud.\nThe drama portrays the reality of Korea through diverse insurance crimes that are prevalent in the country. There are the 'copy cats' that imitate and improve the existing crimes, and the 'mad dogs' who catch these bad cats.",NaN,2,2017
3,3,"[Sulli, Choi Min Ho, Lee Hyun Woo, Kim Ji Won, Seo Joon Young, Kang Ha Neul]","Aug 15, 2012 - Oct 4, 2012",[Jun Ki Sang],64,16,"[Comedy, Romance, School, Youth, Drama, Sports]",7.6,NaN,to the beautiful you,"Goo Jae Hee, is a Korean girl who lives in the United States. One day, she sees a track and field competition on TV, and becomes attracted to one of the high jump competitors, Kang Tae Joon. She begins to idolize the young athlete and in order to help him, she disguises herself as a boy and transfers to his all-male high school after he suffered an accident that could potentially end his career.","[Idol, Cross-dressing, Adapted From A Manga, Celebrity, High Jump]",3,2012
4,4,"[Yeon Jung Hoon, Lee Young Ah, Lee Won Jong, Kim Joo Young, Lee Kyung Young, Gong Jung Hwan]","Sep 9, 2012 - Nov 18, 2012",NaN,65,11,"[Action, Mystery, Vampire, Drama, Investigation]",8.5,[Han Jung Hoon],vampire prosecutor 2,"In the second season of “Vampire Prosecutor,” Min Tae Yeon is back as the prosecutor who heads up a new division that investigates unsolved murder cases. No one, except for Detective Hwang Soon Bum, knows that Tae Yeon was bitten seven years ago and acquired vampire powers. Together, they solve murder cases, and each time Tae Yeon uses his “powers” to find leads on cases that seem unsolvable. But the one thinghe can’t solve is: who was the vampire/serial killer that turned him seven years ago? And what happens when Jung In becomes suspicious of Tae Yeon’s “questionable” tactics in solving cases.",NaN,4,2012
5,5,"[Kim Gab Soo, Im Jung Eun, Jo Hee Bong, Kim Ji Hoon, Jun So Min, Jo Jung Eun]","Aug 20, 2010 - Oct 29, 2010",[Kim Heung Dong],45,12,"[Historical, Mystery, Sci-Fi]",7.8,NaN,joseon x-files,"Secret Investigation Record is based on cases detailed in the Annals of Joseon History with a bizarre twist; these incidents incorporate mysterious, supernatural elements that cannot be explained through science. Kim Hyung Do is an inspector of the state whose single-minded focus on his investigations into administrative misdeeds bring him into direct contact with these strange mysteries. Heo Yoon Yi is a government employee who aids him and attempts to understand thesescientific impossibilities with an open mind, 

#### Theme: tf-idf similarity scores from Dramabeans recaps

In [3]:
# load pickle files for tfidf
version = 'v7-386-bigram'
to_unpickle = ['tfidf', 'index', 'shows', 'corpus', 'dictionary']

tfidf = pickle.load(open('tfidf_files/tfidf_{}.pkl'.format(version), 'rb'))
# index = pickle.load(open('tfidf_files/index_{}.pkl'.format(version), 'rb'))
shows = pickle.load(open('tfidf_files/shows_{}.pkl'.format(version), 'rb'))
corpus = pickle.load(open('tfidf_files/corpus_{}.pkl'.format(version), 'rb'))
dictionary = pickle.load(open('tfidf_files/dictionary_{}.pkl'.format(version), 'rb'))
tfidf_sims = pickle.load(open('tfidf_files/df_sims_{}.pkl'.format(version), 'rb'))

In [4]:
tfidf_sims.head()

,0,1,2,3,4,5,6,7,8,9,...,376,377,378,379,380,381,382,383,384,385
0,0.999999,0.011334,0.009731,0.025026,0.016135,0.015118,0.014226,0.007152,0.008652,0.014048,...,0.016047,0.015826,0.013102,0.014326,0.013595,0.015101,0.013253,0.012696,0.014918,0.012719
1,0.011334,1.000000,0.037899,0.026008,0.024173,0.018407,0.036437,0.017488,0.023477,0.030893,...,0.051181,0.036350,0.037334,0.033350,0.027269,0.034081,0.029075,0.025640,0.025536,0.024713
2,0.009731,0.037899,0.999999,0.025568,0.043958,0.023499,0.045942,0.013299,0.046445,0.040777,...,0.040442,0.044349,0.046795,0.072031,0.031726,0.044627,0.034163,0.047261,0.027057,0.024911
3,0.025026,0.026008,0.025568,1.000000,0.026740,0.021802,0.035824,0.011670,0.022892,0.033981,...,0.034009,0.033820,0.032588,0.028884,0.031461,0.029587,0.033303,0.040008,0.027325,0.043378
4,0.016135,0.024173,0.043958,0.026740,1.000000,0.044416,0.030375,0.014013,0.037894,0.031075,...,0.035333,0.043191,0.046614,0.050390,0.026176,0.036012,0.023292,0.023830,0.021934,0.024227


#### Genres, tags & aired year similarity matrix

In [5]:
# load genres, tags & aired year similarity matrix
genres_sims = pickle.load(open('genres_sim.pkl', 'rb'))
genres_matrix = pickle.load(open('genres_matrix.pkl', 'rb'))
tags_matrix = pickle.load(open('tags_matrix.pkl', 'rb'))
print genres_sims.shape
genres_sims.head()

(386, 386)


,0,1,2,3,4,5,6,7,8,9,...,376,377,378,379,380,381,382,383,384,385
0,1.000000,0.268462,0.220564,0.514650,0.035875,0.024387,0.367301,0.227980,0.082199,0.220564,...,0.234951,0.118108,0.164399,0.025675,0.306236,0.178638,0.170597,0.217383,0.475428,0.427437
1,0.268462,1.000000,0.273861,0.344862,0.207870,0.040373,0.732171,0.641624,0.306186,0.578152,...,0.473523,0.134427,0.317526,0.042505,0.348548,0.480574,0.452856,0.577050,0.499124,0.190516
2,0.220564,0.273861,1.000000,0.200000,0.292770,0.066340,0.387433,0.372104,0.670820,0.400000,...,0.222308,0.455158,0.447214,0.349215,0.312395,0.312395,0.304049,0.387433,0.346984,0.268328
3,0.514650,0.344862,0.200000,1.000000,0.162650,0.022113,0.468997,0.372104,0.223607,0.333333,...,0.305674,0.196343,0.248452,0.023281,0.277684,0.277684,0.261375,0.333056,0.431102,0.387585
4,0.035875,0.207870,0.292770,0.162650,1.000000,0.248174,0.288547,0.302614,0.327327,0.292770,...,0.176272,0.287416,0.218218,0.443039,0.067748,0.237118,0.226445,0.288547,0.015392,0.043644


#### Actor similarity matrix

In [6]:
# load actor similarity matrix
actor_sims = pickle.load(open('actor_sim.pkl', 'rb'))
actor_matrix = pickle.load(open('actor_matrix.pkl', 'rb'))
print actor_sims.shape
actor_sims.head()

(386, 386)


,0,1,2,3,4,5,6,7,8,9,...,376,377,378,379,380,381,382,383,384,385
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0


#### User ratings & crack score

In [7]:
# load user ratings & crack score
ratings = pickle.load(open('user_rating.pkl', 'rb'))
print ratings.shape
ratings.head()

(386, 7)


,showname,dramalist_score,dramabeans_score,av_score,av_score_scaled,crack_raw,crack_scaled
0,ma boy,7.5,8.4,7.95,0.795,273,0.404482
1,falling for innocence,8.4,8.0,8.20,0.820,1092,0.574047
2,mad dog,8.5,9.8,9.15,0.915,998,0.563037
3,to the beautiful you,7.6,6.0,6.80,0.680,1732,0.630467
4,vampire prosecutor 2,8.5,8.0,8.25,0.825,865,0.545543


#### Sentiment category scores from Dramabeans comments

In [8]:
# # sentiment scores
version = 'v4-386'
cat_scores = pickle.load(open('sentiment_files/{}_{}.pkl'.format('all_cat_scores_df', version), 'rb'))
# all_phrases = pickle.load(open('sentiment_files/{}_{}.pkl'.format('comm_vec_phrases', version), 'rb'))
pos_phrases = pickle.load(open('all_pos_phrases.pkl', 'rb'))
sents = pickle.load(open('all_comment_sentences.pkl', 'rb'))
# comm_vec_cat = pickle.load(open('sentiment_files/{}_{}.pkl'.format('comm_vec_cat', version), 'rb'))
# comm_vec = pickle.load(open('sentiment_files/{}_{}.pkl'.format('comm_vec', version), 'rb'))
# # pickle.dump(all_cat_scores_df, open('sentiment_files/{}_{}.pkl'.format('all_cat_scores_df', version), 'wb'))
# pickle.dump(comm_vec, open('sentiment_files/{}_{}.pkl'.format('comm_vec', version), 'wb'))
# pickle.dump(comm_vec_cat, open('sentiment_files/{}_{}.pkl'.format('comm_vec_cat', version), 'wb'))
# pickle.dump(comm_vec_phrases, open('sentiment_files/{}_{}.pkl'.format('comm_vec_phrases', version), 'wb'))

In [9]:
cat_scores.head()

,acting,ending,ent,music,rs,writing_direction
0,10.000000,NaN,NaN,NaN,9.736842,NaN
1,8.200000,6.428571,9.52381,8.181818,7.785714,6.274510
2,7.333333,6.428571,NaN,NaN,7.185185,5.625000
3,8.117647,8.000000,9.60000,6.923077,9.152542,6.774194
4,7.500000,6.923077,NaN,NaN,8.873239,7.500000


In [10]:
# calculate max score and category
cat_scores['max'] = cat_scores.iloc[:,0:6].max(axis=1)
cat_scores['max_scaled'] = cat_scores.iloc[:,0:6].max(axis=1) /10

In [11]:
# identify the category with the top score
def find_cat(row, threshold):
    m = row['max']
    cats = []
    for r in row.index:
        if (m >= threshold) and (row[r] == m) and (r in ['acting', 'ending', 'ent', 'music', 'rs', 'writing_direction']):
#             print r
            if r == 'acting':
                cats.append('Great acting')
            elif r == 'ending':
                cats.append('Satisfying ending')
            elif r == 'ent':
                cats.append('Entertainment value')
            elif r == 'music':
                cats.append('Great soundtrack')
            elif r == 'rs':
                cats.append('Fulfilling relationships')
            elif r == 'writing_direction':
                cats.append('Good writing & direction')
    return ', '.join(cats)

cat_scores['reasons'] = cat_scores.apply(find_cat, threshold=6, axis=1)
cat_scores.head(20)

,acting,ending,ent,music,rs,writing_direction,max,max_scaled,reasons
0,10.000000,NaN,NaN,NaN,9.736842,NaN,10.000000,1.000000,Great acting
1,8.200000,6.428571,9.523810,8.181818,7.785714,6.274510,9.523810,0.952381,Entertainment value
2,7.333333,6.428571,NaN,NaN,7.185185,5.625000,7.333333,0.733333,Great acting
3,8.117647,8.000000,9.600000,6.923077,9.152542,6.774194,9.600000,0.960000,Entertainment value
4,7.500000,6.923077,NaN,NaN,8.873239,7.500000,8.873239,0.887324,Fulfilling relationships
5,9.047619,7.619048,9.000000,NaN,10.000000,6.956522,10.000000,1.000000,Fulfilling relationships
6,7.058824,8.000000,10.000000,7.692308,8.032129,6.750000,10.000000,1.000000,Entertainment value
7,8.571429,NaN,NaN,NaN,NaN,6.500000,8.571429,0.857143,Great acting
8,8.333333,NaN,9.000000,NaN,8.205128,9.523810,9.523810,0.952381,Good writing & direction
9,7.638889,7.200000,NaN,7.692308,7.992832,6.969697,7.992832,0.799283,Fulfilling relationships


In [12]:
terms_dict = {
    'writing_direction' : ['writing', 'pacing', 'storyline', 'plot', 'coherence', 'editing', 'directing', 'direction' 'cinematography'],
    'acting': ['actor', 'actress', 'acting', 'performance', 'cast'],
    'rs': ['romance', 'couple', 'ship', 'bromance', 'chemistry', 'triangle', 'OTP', 'kiss', 'cute', 'friend', 'family'],
    'ending': ['ending', 'resolution', 'finale'],
    'ent': ['entertaining', 'exciting', 'enjoyable', 'thrilling', 'suspense', 'heartwarming'\
                      'crack', 'addictive', 'comedy', 'comedic', 'charm'],
    'music': ['music', 'OST', 'soundtrack', 'sound'] }

In [13]:
# function to pick out the phrases
def find_phrase(ind, terms, pos_phrases= pos_phrases):
    picked_phrases = []
    for keyword in terms:
        for phrase in pos_phrases[ind][0]:
            if keyword in phrase:
                picked_phrases.append(' '.join(phrase))
    return picked_phrases
# find_phrase(2, terms_dict['acting'])

In [33]:
# filter out phrases related to the categories, then randomly pick 10-15 phrases
choice_phrases = []
for ind in cat_scores.index:
    picked_phrases = []
    
    for key in terms_dict.keys():
        picked_phrases += find_phrase(ind, terms_dict[key])

    # filter out duplicates, append to list
    picked_phrases = list(set(picked_phrases))
    
    # filter out phrases that include the word 'recap' & @
    picked_phrases = [phrase for phrase in picked_phrases if ('recap' not in phrase) and ('@' not in phrase) \
                      and ('read' not in phrase) and ('who' not in phrase) and ('their' not in phrase) \
                     and ('(' not in phrase) and (')' not in phrase)]
    
    # pick out a set of phrases randomly picked from the shortlisted ones in picked_phrases
    random_phrases = []
    if len(picked_phrases) >15:
        ids = np.random.choice(range(len(picked_phrases)), size=10, replace=False)
        random_phrases = [picked_phrases[i].capitalize() for i in ids]
    elif len(picked_phrases)>0:
        pp = [' '.join(phrase) for phrase in pos_phrases[ind][0]]
        ids = np.random.choice(range(len(pp)), size=10, replace=False)
        random_phrases = [pp[i].capitalize() for i in ids]
    
#     print random_phrases
    choice_phrases.append(random_phrases)


In [34]:
cat_scores['comments'] = choice_phrases
cat_scores

,acting,ending,ent,music,rs,writing_direction,max,max_scaled,reasons,comments
0,10.000000,NaN,NaN,NaN,9.736842,NaN,10.000000,1.000000,Great acting,"[Time cute twist lol, Favorite plot devices, Decent comedic touch, Hilarious ... love - romance, Real kiss, Drama seems cute, Their chemistry, Light romantic comedy, Cute show in general, Really cute little drama]"
1,8.200000,6.428571,9.523810,8.181818,7.785714,6.274510,9.523810,0.952381,Entertainment value,"[Their chemistry is awesome, Great acting, Pleeeeaseee happy ending, Too cute for words, Sooo adorably cute as min ho, Great family, Hurried overall resolution, Lead couple, Right kiss game is fantastic, Her wonderful acting]"
2,7.333333,6.428571,NaN,NaN,7.185185,5.625000,7.333333,0.733333,Great acting,"[Cute moments, Close friend, Cute the way, Good chemistry, Cute romance, Cute bromance, How subtle the romance, Old friend, Acting is good, Talented actor]"
3,8.117647,8.000000,9.600000,6.923077,9.152542,6.774194,9.600000,0.960000,Entertainment value,"[Cute guy, Best friend, Cute scenes, Cute face, Eun - gyul very cute, Indirect kiss, Excellent acting skill, So cute d, Awesome actor, Sloppy writing]"
4,7.500000,6.923077,NaN,NaN,8.873239,7.500000,8.873239,0.887324,Fulfilling relationships,"[Friend worth, Real romance, Great couple, Bromance was epic with lot of funny, Great pacing, Good actor, Also cute in king, Great actress, Sure the romance, Cast complete]"
5,9.047619,7.619048,9.000000,NaN,10.000000,6.956522,10.000000,1.000000,Fulfilling relationships,"[Actor as intense, Grand finale, Awesome actor, Cool music, Great actor, What a thrilling episode, Playful kiss, Music was okay, Directing is superb, Very low - production comedy]"
6,7.058824,8.000000,10.000000,7.692308,8.032129,6.750000,10.000000,1.000000,Entertainment value,"[Storyline was interesting, Cute moments, Comedic farce, Ggongyangggongyang cute, Entire family, Great couple, Mean friend, Cute innocence, Awesome friend, Namgoongminah chemistry]"
7,8.571429,NaN,NaN,NaN,NaN,6.500000,8.571429,0.857143,Great acting,"[Her acting, Comedy to intense, Soo cute, Better writing, Just good acting, Certain actor, Happy ending, Huge plot holes, So cute together~, Exact storyline]"
8,8.333333,NaN,9.000000,NaN,8.205128,9.523810,9.523810,0.952381,Good writing & direction,"[Ddakji is cute, Comedic relief character, Comedic timing, Amazing cast, Fav actor, Entertaining way 😆, Incidental music, Big love triangle, Quick resolution, Directing are excellent]"
9,7.638889,7.200000,NaN,7.692308,7.992832,6.969697,7.992832,0.799283,Fulfilling relationships,"[Certain otp, Really good actor, Good old couple, Nice chemistry, Unexpectedly cute scene, Official otp, Fast plot, Little found family, Awesome performance, Really cute parts]"


## Combine into scoring matrix

In [35]:
# create version of cat_score for referecing reasons
cat_scores2 = cat_scores.copy()
cat_scores2.index = metadata.showname
# cat_scores2.head()

In [36]:
# reason for recommending
def recc_reason(show):
    showname = show.pop('showname')
    agg = show.pop('aggregate')
#     print show
    reason = [show.argmax()]
    show.drop(show.argmax(), inplace=True)
    reason.append(show.argmax())
    while ('Theme' not in reason) and ('Genre' not in reason) and ('Actors' not in reason) :
        show.drop(show.argmax(), inplace=True)
        reason.append(show.argmax())
    
    # if reason is because of sentiment score, extract out which category is good
    for i, r in enumerate(reason):
#         print i, r
        if r == 'Sentiment':
            reason[i] = cat_scores2.loc[showname, 'reasons']
    return ', '.join(reason)



In [37]:
def topwords(bow, dictionary=dictionary, num_words=50):
    '''displays the top number of words as specified by num_words 
    using the bow (word_id, count/freq)'''
    
    bow_doc = sorted(bow, key=lambda w: w[1], reverse=True)
    words = []

    for word_id, word_count in bow_doc[:num_words]:
        words.append(dictionary.get(word_id))
        
    return words

In [38]:
def find_common_themes(ind1, ind2, tfidf=tfidf, corpus=corpus, dictionary=dictionary, num_words=10):
    '''displays the common themes in the 2 shows with indices ind1 and ind'''
    
    words1 = set(topwords(tfidf[corpus[ind1]]))
    words2 = set(topwords(tfidf[corpus[ind2]]))
    common_themes = ', '.join(list(words1.intersection(words2)))
    
    return 'Similar Theme: ' + common_themes.title()

In [39]:
reccs = {}
for i in metadata.index:

    # combine all the scores into 1 matrix
    comb = pd.concat([metadata.showname, tfidf_sims[i], genres_sims[i], actor_sims[i], ratings[['av_score_scaled', 'crack_scaled']], cat_scores['max_scaled']], axis=1)
    comb.columns = ['showname', 'theme', 'genre', 'actor', 'av_rating', 'crack', 'sentiment']
    
    # drop the selected show
    comb.drop(i, axis=0, inplace=True)

    # impute the missing values in the sentiment scores with the min value, rounded down
    comb.sentiment.fillna(np.round(comb.sentiment.min(), decimals=1), inplace=True)

    # rescale and add weights
    mm = MinMaxScaler()
    comb.theme = mm.fit_transform(comb.theme.values.reshape(-1, 1))
    comb.genre = mm.fit_transform(comb.genre.values.reshape(-1, 1))
    comb.actor = mm.fit_transform(comb.actor.values.reshape(-1, 1))*.75
    comb.av_rating = mm.fit_transform(comb.av_rating.values.reshape(-1, 1))*.8
    comb.crack = mm.fit_transform(comb.crack.values.reshape(-1, 1))*.85
    comb.sentiment = mm.fit_transform(comb.sentiment.values.reshape(-1, 1))*.8

    # rename columns
    comb.columns = ['showname', 'Theme', 'Genre', 'Actors', 'High user rating', 'Addictiveness', 'Sentiment']

    # calculate aggregate score
    best = comb.copy()
    best['aggregate'] = comb.mean(axis=1, skipna=True)
    
    # find reason for recommendation
    best['reason'] = best.apply(recc_reason, axis=1)
    
    # extract top shows, put into dataframe top
    top = best.sort_values('aggregate', ascending=False).head(10)

    # extract details for reason for recommending show
    details = []
    for ind in top.index:
        detailed_reason = []

        # extract common actors
        if top.loc[ind, 'Actors'] != 0:
            common_actors = actor_matrix.loc[[i, ind], :].sum().sort_values(ascending=False)
            common_actors = list(common_actors[common_actors ==2].index)
            detailed_reason.append('Actors in Common: ' + ', '.join(common_actors))
        # extract common genre
        if 'Genre' in top.loc[ind, 'reason']:
            common_genres = genres_matrix.loc[[i, ind], :].sum().sort_values(ascending=False)
            common_genres = list(common_genres[common_genres ==2].index)
            common_tags = tags_matrix.loc[[i, ind], :].sum().sort_values(ascending=False)
            common_tags = list(common_tags[common_tags ==2].index)
            detailed_reason.append('Similar Genre: '+ ', '.join(common_genres + common_tags))
        if 'Theme' in top.loc[ind, 'reason']:
            common_themes = find_common_themes(i, ind)
            detailed_reason.append(common_themes)

        details.append(detailed_reason)
    top['details'] = details
    
    # extract comments for each show
    comments = []
    for ind in top.index:
        comments.append(cat_scores.loc[ind, 'comments'])
    top['comments'] = comments
    
    # put into dictionary
    reccs[i] = top
    
    # print status
    print i,

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [40]:
print metadata.showname[1]
reccs[1]

falling for innocence


,showname,Theme,Genre,Actors,High user rating,Addictiveness,Sentiment,aggregate,reason,details,comments
305,discovery of romance,0.345988,0.999934,0.75,0.533333,0.533160,0.752381,0.652466,"Genre, Entertainment value","[Actors in Common: Yoon Hyun Min, Similar Genre: Romance, Drama, Comedy, Love Triangle]","[Cute voice, Potential romance, Real kiss, Cute part time, Really great chemistry, Open ending writer, Satisfactory ending, Summer romantic comedy, New favorite k - romance, Main plot]"
103,i need romance 3,0.369319,0.773139,0.75,0.590476,0.528303,0.725926,0.622861,"Genre, Actors","[Actors in Common: Kim So Yeon, Similar Genre: Romance, Drama, Comedy]","[Their chemistry really awesome, Cute moments, His cute insecurity, Good chemistry, Amazing chemistry, Softer front sound, Great performance, Lead couple chemistry, Their chemistry so splendid, Most refreshing couple]"
161,"that winter, the wind blows",1.000000,0.766675,0.00,0.622222,0.575623,0.729825,0.615724,"Theme, Genre","[Similar Genre: Romance, Drama, Melodrama, Similar Theme: Secretary]","[Great this plot, Best kiss, Grander plot movements, Initial plot, Loyal best friend, Phenomenal acting, Good chemistry, More straightforward ending, Sad music, Loyal friend]"
48,prosecutor princess,0.471457,0.655898,0.75,0.577778,0.501154,0.729825,0.614352,"Actors, Entertainment value",[Actors in Common: Kim So Yeon],"[Best couple, Fantasized ending, Their chemistry, Debates enjoyable, More entertaining than cinderella, Inhye - happy ending, Awesome writing, Great happy ending conclusion, So cute seo byun, Wonderful couple]"
54,oh my venus,0.317103,0.605446,0.75,0.622222,0.557743,0.657143,0.584943,"Actors, Entertainment value",[Actors in Common: Jo Eun Ji],"[Their bickering was cute, Kiss a little, Meet - cute, Great actress, Plot more interesting, Happy ending wedding, Amazing acting, Silly plot, Their chemistry is great, Good actor]"
200,doctors,0.450787,0.568047,0.75,0.514286,0.560953,0.663492,0.584594,"Actors, Fulfilling relationships",[Actors in Common: Uhm Hyo Sub],"[Acting was superb, Powerful chemistry, Cute little boy, Spell hj charm, General surgery friend, More natural kiss scene, His acting, Awful family, Her kiss face, Cute gangster boss]"
152,beautiful mind,0.383078,0.536524,0.75,0.653968,0.522868,0.632099,0.579756,"Actors, High user rating",[Actors in Common: Yoon Hyun Min],"[Interesting writing, Glad the acting, Shows are enjoyable, Transition from sappy mindless romance, Inner family dynamics, Standard romance, Drama ending nice, Too cute for words, Our favourite wifi couple, Comedic moments]"
139,one more happy ending,0.364580,0.536524,0.75,0.539683,0.523681,0.763964,0.579739,"Entertainment value, Actors",[Actors in Common: Jung Kyung Ho],"[Cute voice, Gel better- plot wise, Longtime friend, More happy ending, Comedic streak, So glad the cute, Real couple, Lovely family, Da - jung , cute, Cute that geun hak]"
10,smile,0.486301,0.401960,0.75,0.596825,0.497137,0.744444,0.579445,"Actors, Entertainment value",[Actors in Common: Jung Kyung Ho],"[Entire kang family, More seasoned actress, More interesting plot, Perfect chemistry, Comedic moment, Real couple, As exciting as others, Couple gloomy, Cute scene, Very cute as siblings]"
166,"hyde, jekyll, me",0.906756,0.831096,0.00,0.507937,0.549915,0.641530,0.572872,"Theme, Genre","[Similar Genre: Romance, Drama, Comedy, Illness, Similar Theme: Secretary]","[Entire cast, Great performance, Intense chemistry, Excellent plot, Plot is obvious, Acting was great, Light sweet romance, Music is good, Nice plot, Smooth plot line]"


In [41]:
# save to pickle (to open in flask!)
pickle.dump(reccs, open('d-reccs-flask.pkl', 'wb'))

### Extract relevant comments from sentiment analysis

In [24]:
terms_dict = {
    'Writing, Direction' : [' writing', ' pacing', 'storyline', ' plot', 'coherence', ' editing', 'directing', 'direction ' 'cinematography'],
    'Acting': ['actor', 'actress', ' acting', 'performance', ' cast'],
    'Relationships': [' romance', 'couple', 'ship ', 'bromance', 'chemistry', 'triangle', 'OTP', 'kiss', 'friendship', 'family'],
    'Ending': [' ending', 'resolution', 'finale'],
    'Overall': ['entertaining', 'exciting', 'enjoyable', 'thrilling', 'suspense', 'heartwarming'\
                      'crack', 'addictive', 'comedy', 'comedic', 'charm'],
    'Music': ['music', 'OST', 'soundtrack'] }

In [25]:
# categorize each comment
def categorize_sents(keyword_list, sents):
    '''categorize each comment in the sents df, remove comments with no tag'''
    sents_df = sents.copy()
    tags = []
    # iterate through each sentence
    for i in sents_df.index:
        category = []
        # iterate through each category
        for cat, keywords in keyword_list.items():
            # iterate through each keyword
            for keyword in keywords:
#                 print keyword
                # check if the keyword exists in the sentence, length is under 80 words, and 'recap' not n the sentence
                if (keyword in sents_df.comments[i].lower()) and (len(sents_df.comments[i].split())< 80) and \
                    ('recap' not in sents_df.comments[i].lower()):
#                     print cat, keyword, sents_df.comments[i].lower()
                    category.append(cat)
#             print 'CATEGORY', category
        tags.append(', '.join(set(category)))
#         print i, 'TAGS', tags
    sents_df['category'] = tags
    sents_df = sents_df[sents_df.category != '']
    sents_df_s = sents_df.sort_values('sent_scores', ascending=False)
    sents_df_s.reset_index(inplace=True, drop=True)
    
    return sents_df_s
            
df = categorize_sents(terms_dict, sents[2])
# print len(tags)
# print sents[0].shape

In [26]:
# for each show, tag each comment & save into a dictionary
comments_dict = {}
for i in metadata.index:
    df = categorize_sents(terms_dict, sents[i])
    comments_dict[i] = df
    print i,
print 'complete!'

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [27]:
# save into pickle
pickle.dump(comments_dict, open('comments_dict.pkl', 'wb'))

In [28]:
# am example of what the comments_dict looks like:
comments_dict[20].head()

,comments,sent_scores,category
0,The premise of that show is nuts & not really politically correct either but I don't care: I'm here for the pure comedy & to see the leads making fun of themselves: The sexy smart guy who's not that smart & the beauty goddess (strutting slow mo included) falling on her face.,0.9666,Overall
1,"You will enjoy this show immensely once you don't buy into what the writer's trying to say, but rather watch for our goofy leads, great chemistry, and the hilariously endearing Ye Seul who is equal parts cute and still retaining some of that charm from Geum Ran.",0.9460,"Relationships, Overall"
2,I like Han Ye-Seul as an talented actress who can look beautiful and act funny at same time.,0.9118,Acting
3,"I love that Tae-hee thinks he’s this smooth operator genius tactician, while the reality is more like he’s half a charming strategist and half a total sham.",0.8955,Overall
4,"Actress who played geum ran was great, and I think ye-seul might end up putting in her best performance of her career in this character (too soon?",0.8934,Acting


### Prepare category scores for flask

In [29]:
# load the category scores matrix
cat_scores = pickle.load(open('sentiment_files/{}_{}.pkl'.format('all_cat_scores_df', version), 'rb'))

# edit so that it's ready to be displayed in flask
cat_scores.columns = ['Acting', 'Ending', 'Entertainment', 'Music', 'Relationships', 'Writing & Direction']
cat_scores[['Acting', 'Writing & Direction', 'Relationships', 'Music', 'Entertainment', 'Ending']]
cat_scores.fillna(0, inplace=True)
cat_scores = cat_scores.applymap(np.round)
cat_scores = cat_scores.astype(int)
cat_scores.head()

# save for flask
pickle.dump(cat_scores, open('cat_scores_flask.pkl', 'wb'))

### Extract show themes from topic modelling

In [30]:
def topwords(bow, dictionary=dictionary, num_words=10):
    '''displays the top number of words as specified by num_words 
    using the bow (word_id, count/freq)'''
    
    bow_doc = sorted(bow, key=lambda w: w[1], reverse=True)
    words = []
    score = []
    for word_id, word_count in bow_doc[:num_words]:
        words.append(dictionary.get(word_id))
        score.append(word_count)
    topwords = pd.DataFrame({'keyword':words, 'score':score}).sort_values('score', ascending=False)
    return topwords

In [31]:
# for each show, collect a list of thematic elements
theme_dict = {}
for show_id in metadata.index:
    
    df = topwords(tfidf[corpus[show_id]], num_words=10)
    lw = list(df.keyword)
    words = []
    # remove duplicates
    for i, v in enumerate(lw):
        lwc = list(lw)
        lwc.pop(i)
        if v not in ', '.join(lwc):
            words.append(v)

    theme_dict[show_id] = words
    
    if show_id % 100 == 0:
        print show_id,
print 'complete!'

0 100 200 300 complete!


In [32]:
# save to pickle for use in flask
pickle.dump(theme_dict, open('theme_dict_flask.pkl', 'wb'))

In [42]:
# example of what it looks like
theme_dict

{0: [u'fanboys',
  u'idol',
  u'crew buddy',
  u'crowd air',
  u'dance crew',
  u'fight dance',
  u'genre market',
  u'lizzie mcguire'],
 1: [u'gold partner',
  u'central factory',
  u'secretary',
  u'product tamper',
  u'bondholder',
  u'new product',
  u'fish watch'],
 2: [u'blackbox',
  u'chief ohns',
  u'original policy',
  u'crash flight',
  u'rest team'],
 3: [u'dorm leader',
  u'soccer',
  u'thump thump',
  u'jumping',
  u'national team',
  u'go thump',
  u'principal',
  u'gymnasium'],
 4: [u'episode',
  u'gangster',
  u'muzzled',
  u'team',
  u'fang',
  u'crime scene',
  u'murder',
  u'aura',
  u'victim',
  u'kidnapper'],
 5: [u'cult',
  u'new land',
  u'shaman',
  u'prophet',
  u'episode',
  u'governor',
  u'ufo',
  u'artifact',
  u'bookshop',
  u'sword'],
 6: [u'gardener',
  u'wig',
  u'bald spot',
  u'president',
  u'attic house',
  u'army friend',
  u'convenience store',
  u'gas station',
  u'seminar'],
 7: [u'new york',
  u'york school',
  u'designer',
  u'seamstress',
  u